https://machinelearningmastery.com/stacking-ensemble-for-deep-learning-neural-networks/
https://machinelearningmastery.com/implementing-stacking-scratch-python/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
import os
import time
import math
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, TensorDataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Get Org Dataset
Obtain the training and test data sets, perform standardization and logarithmic transformation, and verify whether the normalization and logarithmic transformation are successful.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Read the CSV file
org_df = pd.read_csv('/content/drive/MyDrive/CUNY/Capstone/Data/training_data/meta_model_train_test_df.csv')



# Split data into features (X) and target (y)
X = org_df.drop(["Close"], axis=1).copy()
y = org_df["Close"].copy()


# Replace infinite values with NaN
X.replace([np.inf, -np.inf], np.nan, inplace=True)

# Check for infinite values in each column
inf_columns = X.columns[X.isin([np.inf, -np.inf]).any()].tolist()
print("Columns with infinite values:", inf_columns)

# # Fill NaN values with the maximum value of each column
# X.fillna(X.max(), inplace=True)

# Check the count of NaN values in each column
print("NaN counts per column:")
print(X.isnull().sum())

# Scale the features

# Convert the scaled features to a DataFrame
X_df= pd.DataFrame(X, columns=X.columns, index=X.index)

# Combine the target values (y) with the scaled features (X_scaled_df)
org_linear_regression_df = pd.concat([X_df, y], axis=1)



# # Revert transformations
# # 1. Inverse Scaling
# # Restore to original scale
# X_return = scaler.inverse_transform(X_scaled_df)

# # Create a DataFrame for the restored values
# X_return_df = pd.DataFrame(X_return, columns=X.columns, index=X.index)

# linear_regression_return_df = pd.concat([y['Close'], X_return_df], axis=1)

# # 2. Inverse Log Transformation for 'Close'
# linear_regression_return_df['Close'] = np.expm1(org_linear_regression_df['Close'])

# # Create a new dataframe with the original values
# original_values_df = pd.concat([y['Close'], X_return_df], axis=1)

# # Compare to verify the transformations
# comparison_df = original_values_df - linear_regression_return_df

# print(comparison_df.head())


Columns with infinite values: []
NaN counts per column:
SA     0
TA     0
FA1    0
PAS    0
PA     0
CA     0
dtype: int64


In [ ]:
def select_dataset(org_linear_regression_df, org_binary_classification_df, model_type, analysis):

    # Define a dictionary for column selections
    columns_dict = {
        'FA': [
              'Open', 'High', 'Low', 'Vol.', 'Change %', 'Close_delta', 'Open_delta', 'High_delta', 'Low_delta', 'Vol._delta', 'Change %_delta',
              'SOFR', 'HQMCB10YR', 'WM1NS', 'WM2NS', 'CC4WSA', 'CPIAUCSL', 'CSUSHPINSA', 'DCOILWTICO', 'FYFSD', 'GFDEBTN', 'GDPC1', 'ICSA', 'PAYEMS',
              'USSLIND', 'USALOLITONOSTSAM', 'DTWEXBGS', 'DEXUSEU', 'DEXJPUS', 'DEXCHUS', 'DEXUSUK', 'BOGMBASE', 'DGORDER', 'PMI_Actual', 'PMI_Forecast',
              'PMI_Previous', 'VIXCLS', 'Close_delta', 'Open_delta',
              'SOFR_delta', 'HQMCB10YR_delta', 'WM1NS_delta', 'WM2NS_delta', 'CC4WSA_delta', 'CPIAUCSL_delta', 'CSUSHPINSA_delta', 'DCOILWTICO_delta',
              'FYFSD_delta', 'GFDEBTN_delta', 'GDPC1_delta', 'ICSA_delta', 'PAYEMS_delta', 'USSLIND_delta', 'USALOLITONOSTSAM_delta', 'DTWEXBGS_delta',
              'DEXUSEU_delta', 'DEXJPUS_delta', 'DEXCHUS_delta', 'DEXUSUK_delta', 'BOGMBASE_delta', 'DGORDER_delta', 'PMI_Actual_delta', 'PMI_Forecast_delta',
              'PMI_Previous_delta', 'VIXCLS_delta',
              'previous_date_close_1',	'previous_date_close_2',	'previous_date_close_3',	'previous_date_close_4',	'previous_date_close_5'
        ],

        'FA1': [
              'Open', 'High', 'Low', 'Vol.', 'Change %', 'Close_delta', 'Open_delta', 'High_delta', 'Low_delta', 'Vol._delta', 'Change %_delta',
              'SOFR', 'HQMCB10YR', 'WM1NS', 'WM2NS', 'CC4WSA', 'CPIAUCSL', 'CSUSHPINSA', 'DCOILWTICO', 'FYFSD', 'GFDEBTN', 'GDPC1', 'ICSA', 'PAYEMS',
              'USSLIND', 'USALOLITONOSTSAM', 'DTWEXBGS', 'DEXUSEU', 'DEXJPUS', 'DEXCHUS', 'DEXUSUK', 'BOGMBASE', 'DGORDER', 'PMI_Actual', 'PMI_Forecast',
              'PMI_Previous', 'VIXCLS', 'Close_delta', 'Open_delta',
               'VIXCLS_delta',
              'previous_date_close_1',	'previous_date_close_2',	'previous_date_close_3',	'previous_date_close_4',	'previous_date_close_5'
        ],

        'SA': [
        'Open', 'High', 'Low', 'Vol.', 'Change %', 'Close_delta', 'Open_delta', 'High_delta', 'Low_delta', 'Vol._delta', 'Change %_delta',
        'previous_date_close_1',	'previous_date_close_2',	'previous_date_close_3',	'previous_date_close_4',	'previous_date_close_5',
        'negative', 'neutral', 'positive', 'negative_weight', 'neutral_weight', 'positive_weight',
        'SA_simple_score', 'SA_weighted_score',
        'Positive_count', 'Neutral_count', 'Negative_count',
        'positive_score_percentage_simple', 'negative_score_percentage_simple',
        'positive_negative_score_ratio',
        'positive_score_percentage_weighted', 'negative_score_percentage_weighted',
        'positive_negative_score_ratio_weighted',
        'positive_count_percentage', 'negative_count_percentage',
        'positive_negative_count_percentage_ratio',
        'Positive_count_weighted', 'Neutral_count_weighted', 'Negative_count_weighted',
        'positive_count_percentage_weighted', 'negative_count_percentage_weighted',
        'positive_negative_count_percentage_ratio_weighted'
        ],

        'TA': [
        'Open', 'High', 'Low', 'Vol.', 'Change %', 'Close_delta', 'Open_delta', 'High_delta', 'Low_delta', 'Vol._delta', 'Change %_delta',
        'VIXCLS', 'SMA_5', 'SMA_12', 'SMA_26', 'EMA_5', 'EMA_12', 'EMA_26', 'SMA_50', 'SMA_200',
        'macd_12_26_9', 'macd_h_12_26_9', 'macd_s_12_26_9', 'macd_5_9_3', 'macd_h_5_9_3', 'macd_s_5_9_3',
        'macd_24_52_18', 'macd_h_24_52_18', 'macd_s_24_52_18',
        'STOCHk_9_3', 'STOCHd_9_3', 'STOCHk_18_4', 'STOCHd_18_4', 'STOCHk_35_8', 'STOCHd_35_8',
        'RSI_7', 'RSI_14',
        'BB_lower_band_5', 'BB_middle_band_5', 'BB_upper_band_5', 'BB_Width_5', 'BB_Percent_5',
        'BB_lower_band_10', 'BB_middle_band_10', 'BB_upper_band_10', 'BB_Width_10', 'BB_Percent_10',
        'BB_lower_band_20', 'BB_middle_band_20', 'BB_upper_band_20', 'BB_Width_20', 'BB_Percent_20',
        'OBV',
        'ADX_14', 'DMP_14', 'DMN_14', 'ADX_30', 'DMP_30', 'DMN_30', 'ADX_50', 'DMP_50', 'DMN_50',
        'Fib_0.236_5', 'Fib_0.382_5', 'Fib_0.618_5', 'Fib_0.236_20', 'Fib_0.382_20', 'Fib_0.618_20',
        'Fib_0.236_100', 'Fib_0.382_100', 'Fib_0.618_100',
        'CCI_7', 'ROC_7', 'WILLR_7', 'CCI_14', 'ROC_14', 'WILLR_14',
        'VIXCLS_delta', 'SMA_5_delta', 'SMA_12_delta', 'SMA_26_delta', 'EMA_5_delta', 'EMA_12_delta', 'EMA_26_delta',
        'SMA_50_delta', 'SMA_200_delta',
        'macd_12_26_9_delta', 'macd_h_12_26_9_delta', 'macd_s_12_26_9_delta', 'macd_5_9_3_delta', 'macd_h_5_9_3_delta', 'macd_s_5_9_3_delta',
        'macd_24_52_18_delta', 'macd_h_24_52_18_delta', 'macd_s_24_52_18_delta',
        'STOCHk_9_3_delta', 'STOCHd_9_3_delta', 'STOCHk_18_4_delta', 'STOCHd_18_4_delta', 'STOCHk_35_8_delta', 'STOCHd_35_8_delta',
        'RSI_7_delta', 'RSI_14_delta',
        'BB_lower_band_5_delta', 'BB_middle_band_5_delta', 'BB_upper_band_5_delta', 'BB_Width_5_delta', 'BB_Percent_5_delta',
        'BB_lower_band_10_delta', 'BB_middle_band_10_delta', 'BB_upper_band_10_delta', 'BB_Width_10_delta', 'BB_Percent_10_delta',
        'BB_lower_band_20_delta', 'BB_middle_band_20_delta', 'BB_upper_band_20_delta', 'BB_Width_20_delta', 'BB_Percent_20_delta',
        'OBV_delta',
        'ADX_14_delta', 'DMP_14_delta', 'DMN_14_delta', 'ADX_30_delta', 'DMP_30_delta', 'DMN_30_delta', 'ADX_50_delta', 'DMP_50_delta', 'DMN_50_delta',
        'Fib_0.236_5_delta', 'Fib_0.382_5_delta', 'Fib_0.618_5_delta', 'Fib_0.236_20_delta', 'Fib_0.382_20_delta', 'Fib_0.618_20_delta',
        'Fib_0.236_100_delta', 'Fib_0.382_100_delta', 'Fib_0.618_100_delta',
        'CCI_7_delta', 'ROC_7_delta', 'WILLR_7_delta', 'CCI_14_delta', 'ROC_14_delta', 'WILLR_14_delta',
        'Change_abs',
        'previous_date_close_1', 'previous_date_close_2', 'previous_date_close_3', 'previous_date_close_4', 'previous_date_close_5'
        ],

        'CA': [
          'Open', 'High', 'Low', 'Vol.', 'Change %', 'Close_delta', 'Open_delta', 'High_delta', 'Low_delta', 'Vol._delta', 'Change %_delta',
          'far_month_openInterest_0', 'near_month_openInterest', 'total_month_openInterest', 'far_month_openInterest_0_delta', 'near_month_openInterest_delta', 'total_month_openInterest_delta',
          'previous_date_close_1', 'previous_date_close_2', 'previous_date_close_3', 'previous_date_close_4', 'previous_date_close_5'
        ],

        'PA': [
          'Open', 'High', 'Low', 'Vol.', 'Change %', 'Close_delta', 'Open_delta', 'High_delta', 'Low_delta', 'Vol._delta', 'Change %_delta',
          'previous_date_close_1', 'previous_date_close_2', 'previous_date_close_3', 'previous_date_close_4', 'previous_date_close_5'
        ],

        'PAS': [
          'Open', 'High', 'Low', 'Vol.',
        ]

    }

    # Check if the analysis type is valid
    if analysis not in columns_dict:
        raise ValueError(f"Invalid analysis type: {analysis}")

    selected_columns = columns_dict[analysis]

    if model_type == "linear":
        linear_regression_df = org_linear_regression_df[selected_columns + ['Close']]
        return linear_regression_df

    elif model_type == "binary":
        binary_classification_df = org_binary_classification_df[[selected_columns + 'up_down']]
        return binary_classification_df

    else:
        raise ValueError(f"Invalid model type: {model_type}")




In [ ]:
def create_dataset(dataset, lookback):
    X, y = [], []
    for i in range(len(dataset)-lookback):
        feature = dataset[i:i+lookback, :-1]
        target = dataset[i+1:i+lookback+1][-1][-1]
        X.append(feature)
        y.append(target)
    return torch.FloatTensor(X).to(device), torch.FloatTensor(y).view(-1, 1).to(device)


In [ ]:
def loss_curve(epochs, train_loss, test_loss):

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=np.arange(epochs), y=train_loss,
                        mode='lines',
                        name='Train Loss'))
    fig.add_trace(go.Scatter(x=np.arange(epochs) , y=test_loss,
                        mode='lines',
                        name='Validation Loss'))
    fig.update_layout(
        title="Loss curve for single lstm",
        xaxis_title="epochs",
        yaxis_title="rmse"
    )
    fig.show()
# loss_curve(epochs, train_loss, test_loss)

In [ ]:
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler


def predict_plot(model, X_train, X_val, data):

    train_plot = np.ones_like(data[:, 3]) * np.nan
    test_plot = np.ones_like(data[:, 3]) * np.nan




    with torch.no_grad():
        # Predictions on the training set
        y_pred_train = model(X_train)

        train_plot[lookback:int(0.8 * len(data))] = y_pred_train.view(-1).cpu()

        # Predictions on the validation set
        y_pred_val = model(X_val)
        test_plot[int(0.8 * len(data))+lookback:] = y_pred_val.view(-1).cpu()

    train_price = (train_plot * scaler.scale_[0]) + scaler.mean_[0]
    test_price = (test_plot * scaler.scale_[0]) + scaler.mean_[0]
    real_price = (data[:, -1] * scaler.scale_[0]) + scaler.mean_[0]


    # Create the plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=dataset.index, y=train_price,
                            mode='lines',
                            name='Train'))
    fig.add_trace(go.Scatter(x=dataset.index, y=test_price,
                            mode='lines',
                            name='Validation'))
    fig.add_trace(go.Scatter(x=dataset.index, y=real_price,
                            mode='lines',
                            name='True'))

    fig.update_layout(
        title="Stock prediction using single LSTM",
        xaxis_title="Dates",
        yaxis_title="Standardized Stock Price"
    )

    fig.show()

# predict_plot(model, X_train, X_val, data)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import plotly.graph_objects as go

def statistics_index(model, X_val, y_val):

    # Assuming you have the model predictions and target values:
    y_pred = model(X_val)  # Replace with your model's prediction method
    y_true = y_val

    # Move to CPU if they are on GPU:
    if isinstance(y_pred, torch.Tensor) and isinstance(y_true, torch.Tensor):
        y_pred = y_pred.cpu().detach().numpy()
        y_true = y_true.cpu().detach().numpy()

    # Now you can use them with NumPy:
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)

    # Summary
    print(f"\nMSE: {mse}")
    print(f"MAE: {mae}")
    print(f"RMSE: {rmse}")
    print(f"R^2 Score: {r2}")

    return mse, mae, rmse, r2

# Example usage:
# mse, mae, rmse, r2 = statistics_index(model, X_val, y_val)


In [ ]:
import torch
import datetime



def save_model(path, model, model_name):
    datetime_now = datetime.datetime.now()
    est = pytz.timezone('US/Eastern')
    datetime_est = datetime_now.astimezone(est)
    current_time = datetime_est.strftime("%Y%m%d_%H%M%S")
    print(model_name)

    torch.save(model.state_dict(), f"{path}/{model_name}_{current_time}.pth")
    return current_time


In [ ]:
import pandas as pd
import datetime
import pytz

def save_record(csv_file_path):
    # Assuming your existing CSV file is named "S_GRU.csv"
    # csv_file_path = "/content/drive/MyDrive/CUNY/Capstone/Data/model/S_GRU/S_GRU_record.csv"

    # Read the existing CSV file into a DataFrame
    df = pd.read_csv(csv_file_path)

    # Your experimental results, you can replace this with your actual results
    results = {
        'Time': datetime.datetime.now(pytz.timezone('US/Eastern')).strftime("%Y-%m-%d %H:%M:%S"),
        'analyst': 'meta_model',
        'model_name': model_name,
        'model_task': model_task,
        'hidden_size1': hidden_size1,
        'dropout1': dropout1,
        'learning_rate': learning_rate,
        'mse': mse,
        'mae': mae,
        'rmse': rmse,
        'r2': r2,
    }

    # Append the results to the DataFrame
    df = df.append(results, ignore_index=True)

    # Save the updated DataFrame back to the CSV file
    df.to_csv(csv_file_path, index=False)


# Single GRU v1

In [ ]:
import itertools
from tensorflow.keras.callbacks import EarlyStopping

# 建立單層LSTM函式
class S_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size1, dropout1):
        super().__init__()
        self.lstm1 = nn.LSTM(input_size, hidden_size1, num_layers=1, batch_first=True)  # Single LSTM layer
        self.dropout1 = nn.Dropout(dropout1)  # Dropout layer
        self.linear = nn.Linear(hidden_size1, 1)

    def forward(self, x):
        x, _ = self.lstm1(x)
        x = self.dropout1(x)  # Apply dropout
        x = x[:, -1, :]  # Take the last output for prediction
        x = self.linear(x)
        return x
    def hyperparameters(self):
        return {
            'hidden_size1': self.lstm1.hidden_size,  # Access hidden_size instead
            # 'hidden_size2': self.lstm2.hidden_size,  # Commented out as model is single-layer
            'dropout1': self.dropout1.p,
            # 'dropout2': self.dropout2.p,  # Commented out as only one dropout layer
            # Add other hyperparameters as needed
        }

class S_GRU(nn.Module):
    def __init__(self, input_size, hidden_size1, dropout1):
        super().__init__()
        self.lstm1 = nn.GRU(input_size, hidden_size1, num_layers=1, batch_first=True)  # Single LSTM layer
        self.dropout1 = nn.Dropout(dropout1)  # Dropout layer
        self.linear = nn.Linear(hidden_size1, 1)

    def forward(self, x):
        x, _ = self.lstm1(x)
        x = self.dropout1(x)  # Apply dropout
        x = x[:, -1, :]  # Take the last output for prediction
        x = self.linear(x)
        return x
    def hyperparameters(self):
        return {
            'hidden_size1': self.lstm1.hidden_size,  # Access hidden_size instead
            # 'hidden_size2': self.lstm2.hidden_size,  # Commented out as model is single-layer
            'dropout1': self.dropout1.p,
            # 'dropout2': self.dropout2.p,  # Commented out as only one dropout layer
            # Add other hyperparameters as needed
        }



# 建立訓練流程函式
def trainer(epochs, loader, X_train, y_train, X_val, y_val, model, criterion, optimizer):
  train_loss, test_loss = [],[]
  for epoch in range(epochs):
    model.train()
    for batch, (x, y_true) in enumerate(loader):
      y_pred = model(x)
      loss = criterion(y_pred, y_true)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
    model.eval()
    with torch.no_grad():
      y_pred = model(X_train)
      train_rmse = np.sqrt(criterion(y_pred, y_train).item())
      train_loss.append(train_rmse)
      y_pred = model(X_val)
      test_rmse = np.sqrt(criterion(y_pred, y_val).item())
      test_loss.append(test_rmse)
      if (epoch+1) % 100 == 0:
        print('epoch %d train rmse %.4f test rmse %.4f' % (epoch+1, train_rmse, test_rmse))
  return train_loss, test_loss


# Loop

In [ ]:
model_task = 'linear'

dataset = org_linear_regression_df
# 股價
data = dataset.values
# 標準化
scaler = StandardScaler()
data = scaler.fit_transform(data)
# 訓練與驗證集
train, test = data[:int(0.8 * len(data)), :dataset.shape[1]], data[int(0.8 * len(data)):, :dataset.shape[1]]

lookback = 5 # 設定前五天股價預測下一日
X_train, y_train = create_dataset(train, lookback = lookback)
X_val, y_val = create_dataset(test, lookback = lookback)
loader = DataLoader(TensorDataset(X_train, y_train), shuffle = False, batch_size = 32)
feature_amt = X_train.shape[2]



# # Define hyperparameter combinations to explore
# hyperparameters = {
#     'hidden_size1': [128, 256], # 128 256,
#     'dropout1': [0.4, 0.2],
#     'learning_rate': [0.001, 0.005,0.01]
# }



hyperparameters = {
    'hidden_size1': [128, 256], # 128 256,
    'dropout1': [0.4, 0.2],
    'learning_rate': [0.001, 0.005,0.01]
}

patience = 10  # Number of epochs to wait for improvement
early_stopping = EarlyStopping(patience=patience, verbose=True)

# Generate combinations using grid search or random search
combinations = list(itertools.product(*hyperparameters.values()))


best_rmse = float('inf')
best_model = None


for comb in combinations:


    # Unpack hyperparameters
    hidden_size1, dropout1, learning_rate = comb

    # Create the model with hyperparameters from the current combination
    model = S_GRU(input_size=feature_amt, hidden_size1=hidden_size1,  dropout1=dropout1).to(device)
    criterion = nn.MSELoss()


    # Create optimizer with the specified learning rate
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
    epochs = 100

    # Train the model and store RMSE
    train_loss, test_loss = trainer(epochs, loader, X_train, y_train, X_val, y_val, model, criterion, optimizer)
    model_rmse = test_loss[-1]  # Get the RMSE on the validation set from the last epoch

    model_name = f'meta_model_{model.__class__.__name__}_{model_task}_hidden_size1_{hidden_size1}_dropout1_{dropout1}_lr_{learning_rate}'
    mse, mae, rmse, r2 = statistics_index(model, X_val, y_val)
    save_record("/content/drive/MyDrive/CUNY/Capstone/Data/model/S_LSTM_record")

    print(model_name)
    if model_rmse < best_rmse:
        best_rmse = model_rmse
        best_model = model
        loss_curve(epochs, train_loss, test_loss)
        predict_plot(best_model, X_train, X_val, data)


        save_model('/content/drive/MyDrive/CUNY/Capstone/Data/model', best_model, model_name)

        print(best_model.hyperparameters(), "lr= ", learning_rate)




    if early_stopping.stopped_epoch > 0:
        print("Early stopping triggered!")
        break


# 開始訓練並且計算訓練所需時間
start = time.time()
train_loss, test_loss = trainer(epochs, loader, X_train, y_train, X_val, y_val, best_model, criterion, optimizer)
end = time.time()

epoch 100 train rmse 0.0449 test rmse 0.1565

MSE: 0.024488845840096474
MAE: 0.1261705607175827
RMSE: 0.15648911893367767
R^2 Score: 0.8036731148120904
meta_model_S_GRU_linear_hidden_size1_128_dropout1_0.4_lr_0.001


<ipython-input-34-96b0cba3c4b5>:28: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



meta_model_S_GRU_linear_hidden_size1_128_dropout1_0.4_lr_0.001
{'hidden_size1': 128, 'dropout1': 0.4} lr=  0.001
epoch 100 train rmse 0.0724 test rmse 0.1639

MSE: 0.026879392564296722
MAE: 0.13620321452617645
RMSE: 0.16394935548305511
R^2 Score: 0.7845081161446004
meta_model_S_GRU_linear_hidden_size1_128_dropout1_0.4_lr_0.005


<ipython-input-34-96b0cba3c4b5>:28: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



epoch 100 train rmse 0.1510 test rmse 0.1879

MSE: 0.03528854623436928
MAE: 0.16104210913181305
RMSE: 0.18785245716571808
R^2 Score: 0.7170920060664818
meta_model_S_GRU_linear_hidden_size1_128_dropout1_0.4_lr_0.01


...

epoch 100 train rmse 0.1394 test rmse 0.1043

MSE: 0.010882837697863579
MAE: 0.08047830313444138
RMSE: 0.10432083904743195
R^2 Score: 0.9127523820428827
meta_model_S_GRU_linear_hidden_size1_256_dropout1_0.2_lr_0.005


<ipython-input-34-96b0cba3c4b5>:28: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



meta_model_S_GRU_linear_hidden_size1_256_dropout1_0.2_lr_0.005
{'hidden_size1': 256, 'dropout1': 0.2} lr=  0.005
epoch 100 train rmse 0.1245 test rmse 0.3079

MSE: 0.0948193147778511
MAE: 0.28981128334999084
RMSE: 0.3079274594783783
R^2 Score: 0.23983436215663934
meta_model_S_GRU_linear_hidden_size1_256_dropout1_0.2_lr_0.01


<ipython-input-34-96b0cba3c4b5>:28: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



epoch 100 train rmse 0.1394 test rmse 0.1043


In [ ]:
# model_task = 'linear'

# analysts_list = ["FA", "TA", "CA", "PAS","PA",] # SA

# for analyst in analysts_list:

#     analyst = analyst
#     dataset = select_dataset(org_linear_regression_df, org_binary_classification_df, model_task, analyst)
#     # 股價
#     data = dataset.values
#     # 標準化
#     scaler = StandardScaler()
#     data = scaler.fit_transform(data)
#     # 訓練與驗證集
#     train, test = data[:int(0.8 * len(data)), :dataset.shape[1]], data[int(0.8 * len(data)):, :dataset.shape[1]]
#     lookback = 5 # 設定前五天股價預測下一日
#     X_train, y_train = create_dataset(train, lookback = lookback)
#     X_val, y_val = create_dataset(test, lookback = lookback)
#     loader = DataLoader(TensorDataset(X_train, y_train), shuffle = False, batch_size = 32)
#     feature_amt = X_train.shape[2]

#     # Define hyperparameter combinations to explore
#     hyperparameters = {
#         'hidden_size1': [128, 256]
#         'hidden_size2': [8, 16],
#         'dropout1': [0.4,], #0.2
#         'dropout2': [0.2],
#         'learning_rate': [0.001, 0.005,0.01]
#     }
#     patience = 10  # Number of epochs to wait for improvement
#     early_stopping = EarlyStopping(patience=patience, verbose=True)

#     # Generate combinations using grid search or random search
#     combinations = list(itertools.product(*hyperparameters.values()))


#     best_rmse = float('inf')
#     best_model = None


#     for comb in combinations:


#         # Unpack hyperparameters
#         hidden_size1, hidden_size2, dropout1, dropout2, learning_rate = comb

#         # Create the model with hyperparameters from the current combination
#         model = S_GRU(input_size=X_train.shape[2], hidden_size1=hidden_size1,  dropout1=dropout1).to(device)
#         criterion = nn.MSELoss()


#         # Create optimizer with the specified learning rate
#         optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
#         epochs = 1000

#         # Train the model and store RMSE
#         train_loss, test_loss = trainer(epochs, loader, X_train, y_train, X_val, y_val, model, criterion, optimizer)
#         model_rmse = test_loss[-1]  # Get the RMSE on the validation set from the last epoch

#         model_name = f'{analyst}_{model.__class__.__name__}_{model_task}_hidden_size1_{hidden_size1}_dropout1_{dropout1}_lr_{learning_rate}'
#         mse, mae, rmse, r2 = statistics_index(model, X_val, y_val)
#         save_record("/content/drive/MyDrive/CUNY/Capstone/Data/model/S_GRU/S_GRU_record.csv")
#         print(model_name)
#         if model_rmse < best_rmse:
#             best_rmse = model_rmse
#             best_model = model
#             loss_curve(epochs, train_loss, test_loss)
#             predict_plot(best_model, X_train, X_val, data)


#             save_model(best_model, model_name)

#             print(best_model.hyperparameters(), "lr= ", learning_rate)




#         if early_stopping.stopped_epoch > 0:
#             print("Early stopping triggered!")
#             break


#     # 開始訓練並且計算訓練所需時間
#     start = time.time()
#     train_loss, test_loss = trainer(epochs, loader, X_train, y_train, X_val, y_val, best_model, criterion, optimizer)
#     end = time.time()

In [ ]:
# !pip freeze